In [36]:
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np
import BBG_data_process_funcs as data_process
import risk_free_curve as rfcurve
from openpyxl import load_workbook

In [247]:
n = 19
[int(pow(2, b)) for (b, d) in enumerate(reversed(bin(n-1)[2:])) if d == '1']

[2, 16]

In [37]:
%pycat C:/Users/manas/Notebooks/Converts/cbv_models/blackrock_model/AndersonBuffumPricer.py

In [216]:
"""
This file contains functions for risk free curve calibration
"""

import numpy as np
from scipy.interpolate import splev, splrep, splint

# -----------------------------------------------------
# Construct Mark-to-Market Risk Free Discount Curve
# from discrete term structure data
# -----------------------------------------------------

class Libor_Swap_Curve():

    def __init__(self, tenors, zero_rates):
        n_knots_out = 2
        self.knots = np.hstack(
            (np.linspace(0, tenors[0], n_knots_out), tenors[1:-1], np.linspace(tenors[-1], 50, n_knots_out)))
        self.zero_rates_knots = np.hstack(
            (np.ones(n_knots_out) * zero_rates[0], zero_rates[1:-1], np.ones(n_knots_out) * zero_rates[-1]))
        self.tenors = tenors
        self.zero_rates = zero_rates
        self.tck = splrep(self.knots, self.zero_rates_knots)  # B-spline cubic (d=3)
        print(self.knots)
        print(len(tenors))
        print(len(self.knots))
        print(len(splrep(self.knots, self.zero_rates_knots, k = 1)[0]))

    def continuous_zero_curve(self):
        return splev(self.tenors, self.tck)

    def continuous_zero_rate(self, t):
        return splev(t, self.tck)

    def continuous_forward_rate(self, t):
        self.derv = splev(t, self.tck, der=1)
        return splev(t, self.tck) + self.derv * t

    def discount_factor(self, start_t, end_t):
        return np.exp(-splint(start_t, end_t, self.tck))

# -----------------------------------------------------
# class to encapsulate ris.k free rate term structure
# -----------------------------------------------------

class risk_free_curve():

    def __init__(self, tenors, zero_rates):
        self.LSC = Libor_Swap_Curve(tenors, zero_rates)

    def r(self, t):
        return self.LSC.continuous_forward_rate(t)

    def Z(self, t):
        dt = 0.01
        if (t<dt):
            return np.exp(-t*self.r(t))
        else:
            rs = [self.r(s) for s in np.arange(dt,t+1e-4,dt)]
            return np.exp(-dt*np.sum(rs))

In [217]:
pdate = dt.datetime.strptime('2018/09/21', "%Y/%m/%d")
pdate_string = pdate.strftime('%d%b%Y')
inputfile = 'AndersonBuffum_PricingInput_' + pdate_string + 'Final.xlsx'

In [218]:
rf_tenor_tickers = pd.read_excel(inputfile, sheet_name='IR Term TICKERS').set_index('TICKER')
rf_tenor_dict = rf_tenor_tickers['YEARS'].to_dict()

rf_rates_data = pd.read_excel(inputfile, sheet_name='IR Term Structure')
rf_rates_data = rf_rates_data.iloc[6:]
rf_rates_data.columns = np.insert(rf_rates_data.columns[1:],0,'Dates')
rf_rates_data = rf_rates_data.set_index('Dates')

def construct_rf_curve(pricing_date: dt.datetime):
    pdate_rf_rates = rf_rates_data.loc[pdate]
    rf_tenors = np.array([rf_tenor_dict[i] for i in pdate_rf_rates.index])
    rf_zrates = pdate_rf_rates.values/100.
    rf_curve = risk_free_curve(rf_tenors, rf_zrates)
    return rf_curve

rf_curve = construct_rf_curve(pdate)

[ 0.    0.25  0.5   1.    2.    3.    4.    5.    6.    7.    8.    9.
 10.   15.   20.   30.   50.  ]
15
17
19


In [106]:
rf_curve.r(5)

0.03111447529753012

In [12]:
# -----------------------------------------------------
# Function library for Hazard Rate Calibration
# -----------------------------------------------------

import numpy as np
from risk_free_curve import risk_free_curve

# -----------------------------------------------------
# This function requires: Tenors (years), CDS par spread (bps),
# Discount curve (units), Recovery rate (units)
# -----------------------------------------------------

def CDS_Calibration(Tenors, CDS_par_spread, Discount_curve, RR):
    # Vectors to store values:
    dt = np.zeros_like(Tenors)
    P_t = np.zeros(len(Tenors) + 1)
    Lambda_t = np.zeros(len(Tenors))
    P_t[0] = 1

    # Discrete dt:
    dt[0] = Tenors[0]
    dt[1:] = np.diff(Tenors)

    # Define L:
    L = 1 - RR

    # JPMorgan method's Boostrapping of Cumulative survival Prob:
    Summ = 0
    for t, s in enumerate(CDS_par_spread):
        if t == 0:
            P_t[t + 1] = L / (s / 10000.0 * dt[t] + L)
            Lambda_t[t] = -np.log(P_t[t + 1]) / dt[t]
        else:
            Numerator = 0.0
            for i in range(t):
                Numerator += Discount_curve[i] * (L * P_t[i] - (L + dt[i] * s / 10000.0) * P_t[i + 1])

            Denominator = Discount_curve[t] * (L + dt[t] * s / 10000.0)
            P_t[t + 1] = Numerator / Denominator + P_t[t] * L / (L + s / 10000.0 * dt[t])
            Lambda_t[t] = (-np.log(P_t[t + 1]) - np.dot(Lambda_t[:t], dt[:t])) / dt[t]

    return Lambda_t


# -----------------------------------------------------
# class to encapsulate hazard rate term structure
# -----------------------------------------------------


class credit_curve():

    def __init__(self, tenors, spread, recovery_rate, rf_curve: risk_free_curve):
        self.rf_curve = rf_curve
        self.tenors = tenors
        self.spread = spread
        self.rr = recovery_rate
        self._calibrate()

    def _calibrate(self):
        discount_curve = [self.rf_curve.Z(t) for t in self.tenors]
        self.piecewise_hr = CDS_Calibration(self.tenors, self.spread, discount_curve, self.rr)

    def hazard_rate(self, t):
        for i in range(len(self.tenors)):
            if (t <= self.tenors[i]):
                return self.piecewise_hr[i]
        return self.piecewise_hr[-1]

In [ ]:
# ----------------------------------------
# Functions for processing Bloomberg Data
# ----------------------------------------

import numpy as np
import pandas as pd
from ConvertibleBondClass import ConvertibleBond
from AndersonBuffumPricer import AndersenBuffumPricer


# process call put schedule data
def process_call_put_schedule_data(schedule_data):
    ISINs = schedule_data.columns[1::4]
    schedules = {}
    for ISIN in ISINs:
        loc = schedule_data.columns.get_loc(ISIN)
        call = schedule_data.iloc[:,loc:loc+2][1:].dropna()
        call.columns = ['date','price']
        call['date'] = pd.to_datetime(call['date'])
        put = schedule_data.iloc[:,loc+2:loc+4][1:].dropna()
        put.columns = ['date','price']
        put['date'] = pd.to_datetime(put['date'])
        schedules[ISIN] = (call,put)
    return schedules


# parse bloomberg cds spread data
def parse_BBG_cds_spread(ISIN, cds_data):
    try:
        r = cds_data.index[cds_data['ISIN'] == ISIN].tolist()[0] + 1
        data = cds_data.iloc[r]
        sprd = data.values[2:]
        if(any(np.isnan(s) for s in sprd)):
            return np.array([])
        else:
            return sprd
    except:
        return np.array([])


def ABpricer_on_BBG_data(row, pdate, cds_data, cp_schedules, rf_curve, p, use_cds_term_structure=False):
    
    contract_info = {
        'maturity_date': row['MATURITY'],
        'coupon_rate': row['CPN'],
        'coupon_freq': row['CPN_FREQ'],
        'notional': 1000,
        'conv_ratio': None,  # will be derived from conv_price in the code
        'conv_price': row['CV_CNVS_PX'],
        'callable': row['CALLABLE'] == 'Y',
        'puttable': row['PUTABLE'] == 'Y',
    }

    if (contract_info['callable'] or contract_info['puttable']):
        ISIN = row['ISIN']
        if (ISIN in cp_schedules):
            contract_info['call_schedule'], contract_info['put_schedule'] = cp_schedules[ISIN]
        else:
            contract_info['call_schedule'], contract_info['put_schedule'] = (None, None)

    if 'SOFT CALL 20-30' in row:
        contract_info['softcall'] = row['SOFT CALL 20-30'] == 'Y'
    else:
        contract_info['softcall'] = False

    if (contract_info['softcall']):
        contract_info['softcall_start'] = pd.to_datetime(row['SOFT CALL START'])
        contract_info['softcall_end'] = pd.to_datetime(row['SOFT CALL END'])
        contract_info['softcall_barrier'] = row['SOFT CALL BARRIER']
        contract_info['softcall_redempt'] = row['SOFT CALL REDEMPTION']

    rr = row['BOND_RECOVERY_RATE']
    rr = 0.4 if np.isnan(rr) else rr

    if (use_cds_term_structure == True):
        credit_spread = parse_BBG_cds_spread(row['ISIN'], cds_data)
        credit_tenors = [0.5, 1, 2, 3, 4, 5, 7, 10]
    else:
        credit_spread = [row['FLAT_CREDIT_SPREAD_CV_MODEL']]
        credit_tenors = [5]

    model_params = {
        'recovery_rate': rr,
        'equity_spot': row['CV_MODEL_UNDL_PX'],
        'equity_dividend_yield': row['EQY_DVD_YLD_IND'] / 100.,
        'equity_flat_vol': row['CV_MODEL_STOCK_VOL'],
        'eta': row['STOCK_JUMP_ON_DEFAULT_CV_MODEL'],
        'credit tenors': credit_tenors,
        'credit spread': credit_spread, 
    }

    CB = ConvertibleBond(contract_info, model_params, rf_curve, p=p)
    ABpricer = AndersenBuffumPricer(CB, pdate, dt=1. / 48., dy=0.05)

    output = (ABpricer.clean_price(), ABpricer.eq_spot_delta(), ABpricer.eq_spot_gamma(), ABpricer.eq_vega())
    return output


def reformat_output_data(output_data_original, cb_data_input):
    output_data = output_data_original.copy()
    output_data['Issuer'] = cb_data_input['NAME'].values
    output_data['Maturity'] = cb_data_input['MATURITY'].values
    output_data['MKT Price'] = cb_data_input['PX_LAST'].values
    output_data['BBG Price'] = cb_data_input['Fair Value'].values
    output_data['BBG Delta'] = cb_data_input['CV_MODEL_DELTA_V'].values
    output_data['BBG Gamma'] = cb_data_input['CV_MODEL_GAMMA_V'].values
    output_data['BBG Vega'] = cb_data_input['CV_MODEL_VEGA'].values
    output_data['CONV PX'] = cb_data_input['CV_CNVS_PX'].values
    output_data['EQ Spot'] = cb_data_input['CV_MODEL_UNDL_PX'].values
    output_data['EQ Vol'] = cb_data_input['CV_MODEL_STOCK_VOL'].values
    output_data['Flat CDS Spread'] = cb_data_input['FLAT_CREDIT_SPREAD_CV_MODEL'].values

    cols_order = [
        'Issuer', 'Maturity', 'EQ Spot', 'CONV PX', 'EQ Vol', 'Flat CDS Spread',
        'MKT Price', 'BBG Price', 'ABM Price', 'BBG Delta', 'ABM Delta',
        'BBG Gamma', 'ABM Gamma', 'BBG Vega', 'ABM Vega']

    output_data = output_data.loc[:, cols_order]
    output_data['Maturity'] = output_data['Maturity'].astype(str)
    return output_data

In [10]:
import datetime as dt
import numpy as np
from risk_free_curve import risk_free_curve
from cds_function import credit_curve

class ConvertibleBond():

    def __init__(self, contract_info: dict, model_params: dict, rf_curve: risk_free_curve, p=0.0):

        self.p = p  #control elasticity between eq spot and default intensity

        self.m_date = contract_info['maturity_date']
        self.c_rate = contract_info['coupon_rate'] / 100.
        self.c_freq = contract_info['coupon_freq']
        self.notional = contract_info['notional']

        if (('conv_ratio' in contract_info.keys()) and (contract_info['conv_ratio'] != None)):
            self.cv_ratio = contract_info['conv_ratio']
        else:
            self.cv_ratio = self.notional / contract_info['conv_price']

        self.callable = contract_info['callable']
        self.puttable = contract_info['puttable']
        if (self.callable):
            self.call_schedule = contract_info['call_schedule']
        if (self.puttable):
            self.put_schedule = contract_info['put_schedule']

        self.softcall = contract_info['softcall']
        if (self.softcall):
            self.softcallinfo = {
                'start': contract_info['softcall_start'],
                'end': contract_info['softcall_end'],
                'barrier': contract_info['softcall_barrier'],
                'redemption': contract_info['softcall_redempt'] * (self.notional / 100.),  # data was per 100 notional
            }

        # risk free rate term structure
        self.rf_curve = rf_curve

        # hazard rate term structure
        self.rr = model_params['recovery_rate']
        self.ctenors = model_params['credit tenors']
        self.cspread = model_params['credit spread']
        self.cd_curve = credit_curve(self.ctenors, self.cspread, self.rr, self.rf_curve)

        self.eq_spot = model_params['equity_spot']
        self.eq_divd = model_params['equity_dividend_yield']
        self.eq_vol = model_params['equity_flat_vol'] / 100.
        self.eta = model_params['eta']

        # pseudo private variables
        self._y0 = np.log(self.eq_spot)

    def reset_p(self, newp):
        self.p = newp

    def reset_eq_spot(self, new_eq_spot):
        self.eq_spot = new_eq_spot
        self._y0 = np.log(self.eq_spot)

    def r(self, t):
        return self.rf_curve.r(t)

    def Z(self, t):
        return self.rf_curve.Z(t)

    def q(self, t):
        return self.eq_divd

    def lambd(self, t, S):
        y = np.log(S)
        e = self.p * (self._y0 - y)
        h = self.cd_curve.hazard_rate(t)
        return h * np.exp(e)

    def lambd_yspace(self, t, y):
        e = self.p * (self._y0 - y)
        h = self.cd_curve.hazard_rate(t)
        return h * np.exp(e)

    def sigma(self, t):
        return self.eq_vol

    def simulate_eq_spot_paths(self, M, paths, dt):
        np.random.seed(1234567)
        y = np.array([[self._y0] * paths])

        def simulate_next_y(curr_y, t, dt, paths=paths):
            z = np.random.normal(size=paths)
            u = np.random.rand(paths)
            lambd = self.lambd_yspace(t, curr_y)
            drift = (self.r(t) - self.q(t) - (0.5 * self.sigma(t) ** 2) + self.eta * lambd) * dt
            randw = self.sigma(t) * np.sqrt(dt) * z
            jumpN = u <= lambd * dt
            return curr_y + drift + randw - self.eta * jumpN

        for m in range(0, M):
            nexty = simulate_next_y(y[-1], m * dt, dt)
            y = np.vstack((y, nexty))

        return np.exp(y).T  # return paths x T-step matrix

    def soft_call_prob(self, pricing_date, M=20, N=30, paths=1000):
        T2 = (self.softcallinfo['end'] - pricing_date).days / 365.25
        T1 = (self.softcallinfo['start'] - pricing_date).days / 365.25

        dt = 1. / 260  # apply 20-30 soft call logic in business days
        D1 = int(T1 / dt)
        D2 = int(T2 / dt)

        paths = self.simulate_eq_spot_paths(D2, paths, dt)
        self.soft_call_paths = paths[:, D1 - 1:D2 - 1]

        def soft_call_triggered(path, M, N, barrier):
            for i in range(N, len(path)):
                roll_window = path[i - N:i]
                hits = len(roll_window[roll_window >= barrier])
                if (hits >= M):
                    return True
            return False

        b = self.softcallinfo['barrier']
        triggered = np.array([soft_call_triggered(path, M, N, b) for path in self.soft_call_paths])
        soft_call_prob = len(triggered[triggered]) / len(self.soft_call_paths)
        return soft_call_prob

    def soft_call_approx_1of1_barrier(self, pricing_date):
        soft_call_prob = self.soft_call_prob(pricing_date)
        soft_call_paths = self.soft_call_paths
        max_per_path = soft_call_paths.max(axis=1)
        barrier_rank = int(soft_call_prob * len(soft_call_paths))
        barrier_argu = max_per_path.argsort()[-barrier_rank]
        barrier_1of1 = max_per_path[barrier_argu]
        return barrier_1of1

    def fixed_coupon_bond_price(self, pricing_date: dt.datetime):
        pdate_ttm = (self.m_date - pricing_date).days / 365.25
        coupon_dt = 1.0 / float(self.c_freq)
        coupon_ttm = np.arange(0, pdate_ttm + 1e-4, coupon_dt)
        coupon = self.c_rate * coupon_dt * self.notional
        pv_notional = self.notional * self.rf_curve.Z(pdate_ttm)
        pv_coupons = [self.rf_curve.Z(pdate_ttm - tau) * coupon for tau in coupon_ttm]
        return np.sum(pv_coupons) + pv_notional

In [220]:
np.random.rand(4)

array([0.88391701, 0.12693819, 0.34749253, 0.94901707])

In [14]:
# ------------------------------------------------------
# Crank-Nicolson Finite Difference Solver for
# Andersen-Buffum Convertible Bond model
# ------------------------------------------------------

import numpy as np
import datetime as dt
from ConvertibleBondClass import ConvertibleBond

# -----------------------------------------------------
# an efficient tridiagonal-matrix solver
# -----------------------------------------------------

def TDMA(a, b, c, d):
    # tridiagonal-matrix solver: a = Lower Diag, b = Main Diag, c = Upper Diag, d = solution vector
    n = len(d)
    w = np.zeros(n - 1, float)
    g = np.zeros(n, float)
    p = np.zeros(n, float)

    w[0] = c[0] / b[0]
    g[0] = d[0] / b[0]

    for i in range(1, n - 1):
        w[i] = c[i] / (b[i] - a[i - 1] * w[i - 1])
    for i in range(1, n):
        g[i] = (d[i] - a[i - 1] * g[i - 1]) / (b[i] - a[i - 1] * w[i - 1])
    p[n - 1] = g[n - 1]
    for i in range(n - 1, 0, -1):
        p[i - 1] = g[i - 1] - w[i - 1] * p[i]

    return p


# ------------------------------------------------------
# Crank-Nicolson Finite Difference Solver for
# Andersen-Buffum Convertible Bond model
# ------------------------------------------------------

class AndersenBuffumPricer():
    
    theta = 0.5  # keep it as 0.5 to use Crank-Nicolson Method
    y_grid_coverage = 4  # no of stddev covered by the y-grid

    def __init__(self, sec: ConvertibleBond, pdate: dt.datetime, dt=1 / 48, dy=0.05):
        
        self.dt = dt
        self.dy = dy
        self.sec = sec
        self.pdate = pdate

        # discretize time to maturity
        self.ttm = (sec.m_date - pdate).days / 365.25
        self.M = int(self.ttm / dt)  # round down to nearest integer
        self.T = self.M * dt
        self.rf_T = self.ttm - self.T

        # discretize spatial mesh in log-price space
        self.y0 = np.log(sec.eq_spot)
        y_half_range = self.y_grid_coverage * sec.sigma(self.T) * np.sqrt(self.T)
        N_half = int(y_half_range / dy)
        self.N = int(2 * N_half)

        self.t_knots = np.arange(0, self.M + 1, 1) * self.dt
        self.y_knots = self.y0 + np.arange(-N_half, N_half + 0.5, 1) * self.dy
        self.S_knots = np.exp(self.y_knots)
        self.v_knots = np.array([])
        self.S0_index = N_half

        self._init_lambd()
        self._init_sigma_sq()
        self._init_K()
        self._init_L()
        self._init_R()  # recovery value of the bond upon default

        self._init_u()
        self._init_d()
        self._init_l()

        if (self.sec.callable):
            self.callable_grid = self._setup_callable_grid()

        if (self.sec.puttable):
            self.puttable_grid = self._setup_puttable_grid()

        if (self.sec.softcall):
            self.softcall_barrier = self.sec.soft_call_approx_1of1_barrier(pdate)
            self.softcall_barrier_index = np.argmax(self.S_knots >= self.softcall_barrier)
            self.softcall_end_tau = (self.sec.m_date - self.sec.softcallinfo['end']).days / 365.25
            self.softcall_start_tau = (self.sec.m_date - self.sec.softcallinfo['start']).days / 365.25

    def _setup_strike_grid(self, schedule):
        if schedule is None:
            return np.ones(self.M + 1) * self.sec.notional

        dates = schedule['date'].values
        prices = schedule['price'].values * (self.sec.notional / 100.)  # data was per 100 notional
        call_taus = [(self.sec.m_date - d).days / 365.25 for d in dates]
        call_grid = np.zeros(self.M + 1)
        for m in range(0, self.M + 1):
            for call_t, call_price in zip(call_taus, prices):
                if (self.t_knots[m] < call_t and call_t <= self.t_knots[m + 1]):
                    call_grid[m + 1] = call_price

        return call_grid

    def _setup_callable_grid(self):
        return self._setup_strike_grid(self.sec.call_schedule)

    def _setup_puttable_grid(self):
        return self._setup_strike_grid(self.sec.put_schedule)

    def reset_p(self, newp):
        self.sec.reset_p(newp)
        self.__init__(self.sec, self.pdate, self.dt, self.dy)

    def refresh_security(self, sec: ConvertibleBond):
        self.sec = sec
        self.__init__(self.sec, self.pdate, self.dt, self.dy)

    def coupon_stream(self):
        cp_dt = 1.0 / float(self.sec.c_freq)
        cp = self.sec.c_rate * cp_dt * self.sec.notional
        cp_stream = np.zeros(self.M + 1)
        # enumerate in backward time (to be compatible with solver)
        step = int(round(cp_dt / self.dt, 0))
        for m in np.arange(0, self.M + 1, step):
            cp_stream[m] = cp
        return cp_stream

    def accrued_interests(self):
        coupons = self.coupon_stream()
        accrued = np.zeros(self.M + 1)
        c_index = np.argwhere(coupons > 0).flatten()
        c_start = c_index
        c_end = np.append(c_index[1:] - 1, len(coupons) - 2)
        for start, end in zip(c_start, c_end):
            c = coupons[start]
            slots = end - start + 1.
            accrued[start:end + 1] = (c / slots) * np.arange(slots, 0, -1)
        accrued = accrued - coupons
        return accrued

    def initial_condition(self):
        eq = self.sec.cv_ratio * self.S_knots
        bond = self.sec.notional
        return np.maximum(eq, bond) + self.coupon_stream()[0]

    def bc_lower(self):
        recovery = np.array([self.sec.rr * self.sec.notional] * (self.M + 1))
        return recovery + self.accrued_interests()

    def bc_upper(self):
        eq = np.array([self.sec.cv_ratio * self.S_knots[-1]] * (self.M + 1))
        return eq + self.accrued_interests()

    def _init_lambd(self):
        lambd = np.zeros((self.M + 1, self.N + 1))
        for m, tau in np.ndenumerate(self.t_knots):
            t = self.ttm - tau  # tau is backward time here
            lambd[m] = self.sec.lambd(t, self.S_knots)
        self.lambd = lambd

    def _init_sigma_sq(self):
        self.sigma_sq = np.array([self.sec.sigma(self.ttm - tau) ** 2 for tau in self.t_knots])

    def _init_K(self):
        rt = np.array([self.sec.r(self.ttm - tau) for tau in self.t_knots])
        qt = np.array([self.sec.q(self.ttm - tau) for tau in self.t_knots])
        self.K = self.sec.eta * self.lambd + rt[:, np.newaxis] - qt[:, np.newaxis] - 0.5 * self.sigma_sq[:,
                       np.newaxis]  # -----------Warning----------------------

    def _init_L(self):
        rt = np.array([self.sec.r(self.ttm - tau) for tau in self.t_knots])
        self.L = -self.lambd - rt[:, np.newaxis]

    def _init_R(self):
        R = np.ones((self.M + 1, self.N + 1))
        Bond_RR = self.sec.rr * self.sec.notional * R
        EQ_conv = (1 - self.sec.eta) * self.sec.cv_ratio * self.S_knots
        EQ_RR = np.repeat(np.array([EQ_conv]), self.M + 1, axis=0)
        self.R = np.maximum(Bond_RR, EQ_RR)

    def _init_u(self):
        u = self.K + self.sigma_sq[:, np.newaxis] / self.dy
        self.u = (0.5 * self.dt / self.dy) * u

    def _init_d(self):
        d = -self.L + (self.sigma_sq[:, np.newaxis] / (self.dy ** 2))
        self.d = self.dt * d

    def _init_l(self):
        l = self.K - self.sigma_sq[:, np.newaxis] / self.dy
        self.l = (0.5 * self.dt / self.dy) * l

    def solve(self):
        M = self.M
        N = self.N
        theta = self.theta

        coupons = self.coupon_stream()
        accrued = self.accrued_interests()
        eq_conv = self.sec.cv_ratio * self.S_knots

        bc_lower = self.bc_lower()
        bc_upper = self.bc_upper()
        v = self.initial_condition()  # (N+1)-array
        v_dt = self.initial_condition()  # (N+1)-array

        for m in range(0, M):
            z = (1 - theta) * (self.u[m, 1:-1] * v[2:])
            z += (1 - (1 - theta) * self.d[m, 1:-1]) * v[1:-1]
            z -= (1 - theta) * self.l[m, 1:-1] * v[0:-2]
            z += self.dt * theta * (self.lambd[m + 1, 1:-1] * self.R[m + 1, 1:-1])
            z += self.dt * (1 - theta) * (self.lambd[m, 1:-1] * self.R[m, 1:-1])

            v[0] = bc_lower[m + 1]
            # v[N] = bc_upper[m+1]
            v[N] = 2 * v[N - 1] - v[N - 2]

            b = np.zeros(N - 1)
            b[0] = -theta * self.l[m + 1, 1] * v[0]
            b[-1] = theta * self.u[m + 1, N - 1] * v[N]

            C_l_diag = theta * self.l[m + 1, 2:-1]
            C_m_diag = 1 + theta * self.d[m + 1, 1:-1]
            C_u_diag = -theta * self.u[m + 1, 1:-2]

            # solve tridiagonal-matrix problem
            v[1:-1] = TDMA(C_l_diag, C_m_diag, C_u_diag, z + b)

            # 20-of-30 soft-call by issuer
            if (self.sec.softcall):
                tau = (m + 1) * self.dt
                if (tau >= self.softcall_end_tau and tau <= self.softcall_start_tau):
                    barx = self.softcall_barrier_index
                    v[barx:-1] = np.minimum(v[barx:-1], self.sec.softcallinfo['redemption'] + accrued[m + 1])

            # callable option by issuer has the least priority (iterating backward here)
            if (self.sec.callable and self.callable_grid[m + 1] > 0):
                call_price = self.callable_grid[m + 1]
                v[1:-1] = np.minimum(v[1:-1], call_price + accrued[m + 1])

            # puttable option by borrower (priority over issuer's call)
            if (self.sec.puttable and self.puttable_grid[m + 1] > 0):
                put_price = self.puttable_grid[m + 1]
                v[1:-1] = np.maximum(v[1:-1], put_price + accrued[m + 1])

                # borrower has option to convert any time (priority over issuer's call)
            v[1:-1] = np.maximum(v[1:-1], eq_conv[1:-1] + accrued[m + 1])

            # add discrete coupons - coupons are always paid before any conversion/call/put
            v[1:-1] = v[1:-1] + coupons[m + 1]

        # discount by the risk-free rate for the residual period smaller than dt
        self.v_knots = v * self.sec.Z(self.rf_T)

    def price(self):
        if (len(self.v_knots) == 0):
            self.solve()
        return self.v_knots[self.S0_index]

    def dirty_price(self):
        p = self.price()
        return p * (100. / self.sec.notional)

    def clean_price(self):
        dt = 1. / float(self.sec.c_freq)
        cp = self.sec.c_rate * dt * 100  # coupon per 100 notional
        remain = int(self.ttm / dt)
        accrued_t = dt - (self.ttm - remain * dt)
        accrued_c = cp * (accrued_t / dt)
        return self.dirty_price() - accrued_c

    def eq_spot_delta(self):
        self.solve()
        dvdy = (self.v_knots[self.S0_index + 1] - self.v_knots[self.S0_index - 1]) / (2 * self.dy)
        dvds = dvdy / self.sec.eq_spot
        delta = dvds / self.sec.cv_ratio  # same convention as  Bloomberg and BlackRock
        return delta

    def eq_spot_gamma(self):
        base_spot = self.sec.eq_spot
        abs_shock = min(10.0 / self.sec.cv_ratio, 0.01 * base_spot)

        self.sec.reset_eq_spot(base_spot + abs_shock)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        up_delta = self.eq_spot_delta()

        self.sec.reset_eq_spot(base_spot - abs_shock)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        dn_delta = self.eq_spot_delta()

        # revert to the base state
        self.sec.reset_eq_spot(base_spot)
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        return (up_delta - dn_delta) / (2 * abs_shock / base_spot)

    def eq_vega(self, shock=0.01):
        base_vol = self.sec.eq_vol

        self.sec.eq_vol = base_vol + shock
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        shocked_price = self.dirty_price()

        self.sec.eq_vol = base_vol
        self.__init__(self.sec, self.pdate, dt=self.dt, dy=self.dy)
        base_price = self.dirty_price()

        vega = (shocked_price - base_price)
        return vega